# Digit Recognition Using PyTorch (ANN and CNN Implementation)

#### Just learnt and gone through how things work in PyTorch, how to train and work on deep learning use cases, and thouht to try out my skills on this baseline model.

#### All steps are covered from preparing data loaders to training and Validating our model. Feel free to give feedback.



## I am just a beginner and your upvote will motivate me! 🤗😇 

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


#PyTorch
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch import optim , nn
import torch.nn.functional as F
from torch.utils.data import Dataset

## Preparing Data

In [ ]:
#Getting csv data

train = pd.read_csv('../input/digit-recognizer/train.csv')
x_train = train.iloc[:,1:].values/255
y_train = train.label.values

## Preparing Train and Validation Data

In [ ]:
#Dividing into train and validation set

train_x, val_x, train_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
#Converting into Tensors from_numpy()

train_x_torch = torch.from_numpy(train_x).type(torch.FloatTensor)
val_x_torch = torch.from_numpy(val_x).type(torch.FloatTensor)

train_y_torch = torch.from_numpy(train_y).type(torch.LongTensor) #Data typecasting
val_y_torch = torch.from_numpy(val_y).type(torch.LongTensor)

In [ ]:
batch_size = 128  #anything b/w 64 and 256 works

#Preparing training set and test set
trainset = torch.utils.data.TensorDataset(train_x_torch, train_y_torch)
valset = torch.utils.data.TensorDataset(val_x_torch, val_y_torch)


#Preparing Data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
valloader = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)

# 1. ANN (Artificial Neural Network)

## Constructing a basic Neural Network in PyTorch

In [ ]:
# Building a Neural Network

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28*1, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,64)
        self.fc6 = nn.Linear(64,10)
        
        self.dropout = nn.Dropout(p=0.3)
        
        self.softmax = F.log_softmax
        
    def forward(self, X):
        X = self.dropout(F.relu(self.fc1(X)))
        X = self.dropout(F.relu(self.fc2(X)))
        X = self.dropout(F.relu(self.fc3(X)))
        X = self.dropout(F.relu(self.fc4(X)))
        X = self.dropout(F.relu(self.fc5(X)))
        X = self.softmax(self.fc6(X), dim=1)
        
        return X

## Training and Validation

In [ ]:
## Training Our Neural Network
model = Net()

def fit(model, trainloader, valloader, epochs = 25):
    criterion = nn.NLLLoss()

    #weight decay for L2 Regularization
    optimizer = optim.Adam(model.parameters(), lr = 0.0003)

    epochs = epochs

    steps = 0
    print_at = 50

    train_losses, test_losses = [], []

    for e in range(epochs):
        running_loss = 0
    
        for images, labels in trainloader:
            steps+=1
        
            #Start from zero every epoch
            optimizer.zero_grad()
        
            #Make predictions
            output = model(images)
        
            #Calculate loss
            loss = criterion(output, labels)
        
            #backprop
            loss.backward()
        
            #Adjusting weights
            optimizer.step()
        
            running_loss += loss.item()
        
            #For validation
            if steps%print_at == 0:
                test_loss = 0
                accuracy = 0
            
                #Turn of gradients and go into eval mode
                with torch.no_grad():
                    model.eval()
                
                    for images, labels in valloader:
                        output = model(images)
                        test_loss += criterion(output, labels)
                    
                        probs = torch.exp(output)
                    
                        top_p, top_class = probs.topk(1, dim = 1)
                    
                        equals = top_class == labels.view(*top_class.shape)
                    
                        accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
                model.train()
            
                train_losses.append(running_loss/len(trainloader))
                test_losses.append(test_loss/len(valloader))
            
                print('epochs{}/{}.. '.format(e+1, epochs),
                     "Training Loss: {:.3f}.. ".format(train_losses[-1]),
                     "Validation Loss: {:.3f}.. ".format(test_losses[-1]),
                     "Test Accuracy: {:.3f}.. ".format(accuracy/len(valloader)) )
            
    return train_losses, test_losses       

In [ ]:
train_losses, test_losses = fit(model, trainloader, valloader)

## How did our baseline model performed

In [ ]:
%matplotlib inline

plt.plot(train_losses, label ='Training Loss')
plt.plot(test_losses, label = 'Test Loss')
plt.title('ANN Training')
plt.legend(frameon=False)

## Preparing test Data for submission

In [ ]:
test = pd.read_csv('../input/digit-recognizer/test.csv')
x_test = test.values/255
x_test_torch = torch.from_numpy(x_test).type(torch.FloatTensor)

In [ ]:
dummy_labels = np.zeros(x_test.shape)
dummy_labels = torch.from_numpy(dummy_labels)

In [ ]:
testset = torch.utils.data.TensorDataset(x_test_torch, dummy_labels)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

## Making Predictions

In [ ]:
submit = [['ImageId', 'Label']]

#turning of gradients
with torch.no_grad():
    model.eval()
    image_id = 1
    
    for images, _ in testloader:
        outputs = model(images)
        probs = torch.exp(outputs)
        
        top_p, top_class = probs.topk(1, dim = 1)
        
        for preds in top_class:
            submit.append([image_id,preds.item()])
            image_id += 1

## Creating ANN Submission File!

In [ ]:
submit_df = pd.DataFrame(submit)
submit_df.columns = submit_df.iloc[0]
submit_df = submit_df.drop(0, axis = 0)

submit_df.to_csv('ANN_Submission.csv', index = False)

# 2. CNN (Convolution Neural Network)

In [ ]:
batch_size = 128

train_x_torch = train_x_torch.view(-1, 1,28,28).float()
val_x_torch = val_x_torch.view(-1, 1,28,28).float()

#preparing training and validation dataset
trainset = torch.utils.data.TensorDataset(train_x_torch, train_y_torch)
valset = torch.utils.data.TensorDataset(val_x_torch, val_y_torch)

#preparing Data Loaders

trainloader = torch.utils.data.DataLoader(trainset, shuffle = True, batch_size = batch_size)
valloader = torch.utils.data.DataLoader(valset, shuffle = True, batch_size = batch_size)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size = 3, padding =1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 3, padding =1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size = 3, padding =1)
        
        self.relu = nn.ReLU()
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(3*3*128, 512)
        
        self.fc2 = nn.Linear(512, 10)
        
        self.dropout = nn.Dropout(p=0.3)
        
        self.softmax = F.log_softmax
        
    def forward(self, X):
        X = self.dropout(self.relu(self.pool(self.conv1(X))))
        X = self.dropout(self.relu(self.pool(self.conv2(X))))
        X = self.dropout(self.relu(self.pool(self.conv3(X))))
        X = X.view(-1, 3*3*128)
        
        X = self.dropout(self.relu(self.fc1(X)))
        
        X = self.softmax(self.fc2(X), dim = 1)
        
        return X
        
        

In [ ]:
cnn = CNN()

#training our CNN
train_losses, test_losses = fit(cnn, trainloader, valloader, epochs = 10)

In [ ]:
%matplotlib inline

plt.plot(train_losses, label ='Training Loss')
plt.plot(test_losses, label = 'Test Loss')
plt.title('CNN Training')
plt.legend(frameon=False)

In [ ]:
#reshaping test data for feeding CNN
x_test_torch = x_test_torch.view(-1, 1, 28, 28)

testset = torch.utils.data.TensorDataset(x_test_torch, dummy_labels)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

In [ ]:
submit = [['ImageId', 'Label']]

#turning of gradients
with torch.no_grad():
    cnn.eval()
    image_id = 1
    
    for images, _ in testloader:
        outputs = cnn(images)
        probs = torch.exp(outputs)
        
        top_p, top_class = probs.topk(1, dim = 1)
        
        for preds in top_class:
            submit.append([image_id,preds.item()])
            image_id += 1

## Making CNN Submission file!

In [ ]:
submit_df = pd.DataFrame(submit)
submit_df.columns = submit_df.iloc[0]
submit_df = submit_df.drop(0, axis = 0)

submit_df.to_csv('CNN_Submission.csv', index = False)